In [82]:
import pyspark
from pyspark.sql import SparkSession
import os

os.environ["JAVA_HOME"] = "/home/nico/DataZoomCamp2024/week5-batch-spark/spark/jdk-11.0.2"
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [83]:
spark.version

'3.5.1'

In [84]:
df = spark.read \
    .option("header", "true") \
    .csv('fhv_tripdata_2019-10.csv')

df = df.repartition(6)

df.write.parquet('data/pq/fhv/')

In [85]:
 ls -la ./data/pq/fhv/

total 39168
drwxr-xr-x 2 nico nico    4096 Feb 27 15:12 ./
drwxr-xr-x 4 nico nico    4096 Feb 27 15:12 ../
-rw-r--r-- 1 nico nico       8 Feb 27 15:12 ._SUCCESS.crc
-rw-r--r-- 1 nico nico   51812 Feb 27 15:12 .part-00000-c6cc2dd6-3ee0-4910-a3c0-49db26b95444-c000.snappy.parquet.crc
-rw-r--r-- 1 nico nico   51736 Feb 27 15:12 .part-00001-c6cc2dd6-3ee0-4910-a3c0-49db26b95444-c000.snappy.parquet.crc
-rw-r--r-- 1 nico nico   51816 Feb 27 15:12 .part-00002-c6cc2dd6-3ee0-4910-a3c0-49db26b95444-c000.snappy.parquet.crc
-rw-r--r-- 1 nico nico   51800 Feb 27 15:12 .part-00003-c6cc2dd6-3ee0-4910-a3c0-49db26b95444-c000.snappy.parquet.crc
-rw-r--r-- 1 nico nico   51808 Feb 27 15:12 .part-00004-c6cc2dd6-3ee0-4910-a3c0-49db26b95444-c000.snappy.parquet.crc
-rw-r--r-- 1 nico nico   51756 Feb 27 15:12 .part-00005-c6cc2dd6-3ee0-4910-a3c0-49db26b95444-c000.snappy.parquet.crc
-rw-r--r-- 1 nico nico       0 Feb 27 15:12 _SUCCESS
-rw-r--r-- 1 nico nico 6630585 Feb 27 15:12 part-00000-c6cc2dd6-3ee0-4910-a3c0-4

In [86]:
from pyspark.sql import functions as F
df.withColumn('pickup_date', F.to_date(df.pickup_datetime)).filter("pickup_date = '2019-10-15'").count()

62610

In [87]:
df.registerTempTable('fhv')
df.columns

/home/nico/anaconda3/lib/python3.11/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


['dispatching_base_num',
 'pickup_datetime',
 'dropOff_datetime',
 'PUlocationID',
 'DOlocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [88]:
df_zones = spark.read.option("header", True).csv('taxi_zone_lookup.csv')
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [89]:
df_zones.registerTempTable('zones')

In [90]:
spark.sql("""
SELECT
    count(PUlocationID),
    zones.Zone
FROM 
    fhv
join zones on fhv.PULocationID = zones.LocationID
GROUP BY
    zones.Zone
ORDER BY
    count(PUlocationID) ASC
LIMIT 1;
""").show()

+-------------------+-----------+
|count(PUlocationID)|       Zone|
+-------------------+-----------+
|                  1|Jamaica Bay|
+-------------------+-----------+

